In [ ]:
import scipy as sp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import h5py
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.inspection import permutation_importance
import xarray as xr
import statistics as st
import random
import warnings
# Import tensorflow and keras
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats, odr
#import shap

from sklearn.model_selection import train_test_split

import matplotlib as mpl

In [ ]:
################################################################################
# Need to use "IPWG Extraction Script" here prior to running any further cells #
################################################################################

In [ ]:
def my_loss(y_true, y_pred):
    # term_1 = tf.math.pow(tf.math.maximum(tf.math.abs(y_true), tf.math.abs(y_pred)), 5.0)  # symmetric
    term_1 = tf.math.pow(tf.math.abs(tf.math.maximum(y_true, y_pred)), 10.0)  # asymmetric
    term_2 = tf.math.pow((y_true-y_pred), 2.0)
    loss_value = tf.multiply(term_1, term_2)
    return tf.math.reduce_mean(loss_value)
print('Did it work?')

In [ ]:
def build_model(x_train, y_train, settings):
    # create input layer
    input_layer = tf.keras.layers.Input(shape=x_train.shape[1:])

    # create a normalization layer if you would like
    normalizer = tf.keras.layers.Normalization(axis=(1,))
    normalizer.adapt(x_train)
    layers = normalizer(input_layer)

    # create hidden layers each with specific number of nodes
    assert len(settings["hiddens"]) == len(
        settings["activations"]
    ), "hiddens and activations settings must be the same length."

    # add dropout layer
    layers = tf.keras.layers.Dropout(rate=settings["dropout_rate"])(layers)

    for hidden, activation in zip(settings["hiddens"], settings["activations"]):
        layers = tf.keras.layers.Dense(
            units=hidden,
            activation=activation,
            use_bias=True,
            kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0, l2=0),
            bias_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"]),
            kernel_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"]),
        )(layers)

    # create output layer
    output_layer = tf.keras.layers.Dense(
        units=1,  # one unit in the output since this is a regression problem
        activation="linear",
        use_bias=True,
        bias_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"] + 1),
        kernel_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"] + 2),
    )(layers)

    # construct the model
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.summary()

    return model


def compile_model(model, settings):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=settings["learning_rate"]),
        loss="mae",
        # loss = my_loss,
        metrics=[
            "mse",
            "mae"
        ],
    )
    return model
print('Did it work?')

In [ ]:
# Split the data into training and testing sets

# Tunable Parameter: Describes the proportion of the dataset we want to use for testing. 1 - split_size is used for training.
split_size = 0.2

# PARAMETERS:
#     test_size: fraction of testing/validation datasets
#     random_state: random parameter
Xtrain, Xval, Ytrain, Yval = train_test_split(
    input_raw, output_raw, test_size=split_size, random_state=42
)
print('Did it work?')

In [ ]:
settings = {
    "hiddens": [3, 3],
    "activations": ["relu", "relu"],
    "learning_rate": 0.0025,
    "random_seed": 33,
    "max_epochs": 5_000,
    "batch_size": 256,
    "patience": 10,
    "dropout_rate": 0.,
}

tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(settings["random_seed"])

model = build_model(Xtrain, Ytrain, settings)
model = compile_model(model, settings)
print('Did it work?')

In [ ]:
# define the early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=settings["patience"], restore_best_weights=True, mode="auto", verbose=1,
)

# train the model via model.fit
history = model.fit(
    Xtrain,
    Ytrain,
    epochs=settings["max_epochs"],
    batch_size=settings["batch_size"],
    shuffle=True,
    validation_data=[Xval, Yval],
    callbacks=[early_stopping_callback],
    verbose=2,
)
print('Did it work?')

In [ ]:
# Let's plot the change in loss and categorical_accuracy

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

axs[0].plot(history.history["loss"], label="training")
axs[0].plot(history.history["val_loss"], label="validation")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].legend()

axs[1].plot(history.history["mae"], label="training")
axs[1].plot(history.history["val_mae"], label="validation")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Mean Absolute Error")
axs[1].legend()
plt.show()

In [ ]:
###########################################################
#Training same model as above but with the criteria as MSE#
###########################################################

In [ ]:
def my_loss(y_true, y_pred):
    # term_1 = tf.math.pow(tf.math.maximum(tf.math.abs(y_true), tf.math.abs(y_pred)), 5.0)  # symmetric
    term_1 = tf.math.pow(tf.math.abs(tf.math.maximum(y_true, y_pred)), 10.0)  # asymmetric
    term_2 = tf.math.pow((y_true-y_pred), 2.0)
    loss_value = tf.multiply(term_1, term_2)
    return tf.math.reduce_mean(loss_value)
print('Did it work?')

In [ ]:
def build_model(x_train, y_train, settings):
    # create input layer
    input_layer = tf.keras.layers.Input(shape=x_train.shape[1:])

    # create a normalization layer if you would like
    normalizer = tf.keras.layers.Normalization(axis=(1,))
    normalizer.adapt(x_train)
    layers = normalizer(input_layer)

    # create hidden layers each with specific number of nodes
    assert len(settings["hiddens"]) == len(
        settings["activations"]
    ), "hiddens and activations settings must be the same length."

    # add dropout layer
    layers = tf.keras.layers.Dropout(rate=settings["dropout_rate"])(layers)

    for hidden, activation in zip(settings["hiddens"], settings["activations"]):
        layers = tf.keras.layers.Dense(
            units=hidden,
            activation=activation,
            use_bias=True,
            kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0, l2=0),
            bias_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"]),
            kernel_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"]),
        )(layers)

    # create output layer
    output_layer = tf.keras.layers.Dense(
        units=1,  # one unit in the output since this is a regression problem
        activation="linear",
        use_bias=True,
        bias_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"] + 1),
        kernel_initializer=tf.keras.initializers.RandomNormal(seed=settings["random_seed"] + 2),
    )(layers)

    # construct the model
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.summary()

    return model


def compile_model(model, settings):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=settings["learning_rate"]),
        loss="mse",
        # loss = my_loss,
        metrics=[
            "mse",
            "mae"
        ],
    )
    return model
print('Did it work?')

In [ ]:
settings = {
    "hiddens": [3, 3],
    "activations": ["relu", "relu"],
    "learning_rate": 0.0025,
    "random_seed": 33,
    "max_epochs": 5_000,
    "batch_size": 256,
    "patience": 10,
    "dropout_rate": 0.,
}

tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(settings["random_seed"])

model = build_model(Xtrain, Ytrain, settings)
model = compile_model(model, settings)
print('Did it work?')

In [ ]:
# define the early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=settings["patience"], restore_best_weights=True, mode="auto", verbose=1,
)

# train the model via model.fit
history = model.fit(
    Xtrain,
    Ytrain,
    epochs=settings["max_epochs"],
    batch_size=settings["batch_size"],
    shuffle=True,
    validation_data=[Xval, Yval],
    callbacks=[early_stopping_callback],
    verbose=2,
)
print('Did it work?')

In [ ]:
# Let's plot the change in loss and categorical_accuracy

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

axs[0].plot(history.history["loss"], label="training")
axs[0].plot(history.history["val_loss"], label="validation")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].legend()

axs[1].plot(history.history["mae"], label="training")
axs[1].plot(history.history["val_mae"], label="validation")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Mean Absolute Error")
axs[1].legend()
plt.show()

In [ ]:
#######################################################
# Pick the better model or do the following with both #
#######################################################

In [ ]:
########################
# Read in Testing Data #
########################

In [ ]:
# What predictions did the model make for our training, validation, and test sets?
Ptrain = model.predict(Xtrain)  # Array of predictions
Pval = model.predict(Xval)
Ptest = model.predict(Xtest)
print('Did it work?')

In [ ]:
error_test = np.abs(Pval - Yval)[:,0]
plt.figure()
plt.plot(Yval, error_test, ".")
plt.xlabel("true value")
#plt.ylabel("predicted value")
plt.ylabel("error_test")
plt.show()
#features = np.array(data_tbu[INPUT_VARIABLES])
#feature_list = list(features.columns)

In [ ]:
error_test_t = np.abs(Ptest - Ytest)[:,0]
plt.figure()
plt.plot(Ytest, error_test_t, ".")
plt.xlabel("true value")
#plt.ylabel("predicted value")
plt.ylabel("error_test")
plt.show()
#features = np.array(data_tbu[INPUT_VARIABLES])
#feature_list = list(features.columns)

In [ ]:
plt.plot(Yval, Yval, 'bo', label = 'tbs_atms')
plt.plot(Yval, Pval, 'rx', label = 'tbs_atms')
plt.show()

In [ ]:
plt.plot(Ytest, Ytest, 'bo', label = 'tbs_atms')
plt.plot(Ytest, Ptest, 'rx', label = 'tbs_atms')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

MSEtr = mean_squared_error(Ytrain,Ptrain)
MAEtr = mean_absolute_error(Ytrain,Ptrain)
MSEv = mean_squared_error(Yval,Pval)
MAEv = mean_absolute_error(Yval,Pval)
MSEt = mean_squared_error(Ytest,Ptest)
MAEt = mean_absolute_error(Ytest,Ptest)

#print('Training MSE : ', MSEtr)
#print('Training MAE : ', MAEtr)
print('-----------------------------------')
print('Validation MSE : ', MSEv)
print('Validation MAE : ', MAEv)
print('-----------------------------------')
print('Testing MSE : ', MSEt)
print('Testing MAE : ', MAEt)
print('-----------------------------------')